# Exercises & Extra-curriculum

## 1. Turn the code to get the data (from section 1. Get Data above) into a Python script, such as `get_data.py`
- When you run the script using `python get_data.py` it should check if the data already exists and skip downloading if it does.
- If the data download is successful, you should be able to access the `pizza_steak_sushi` images from the data directory.

In [1]:
import os

os.makedirs('going_modular', exist_ok=True)

In [2]:
%%writefile going_modular/get_data.py

# Import libraries
import os
import requests
import zipfile
import torch
import torchvision
from torchvision import datasets
from pathlib import Path

# Setup path to data folder
data_path = Path("data/")
image_path = data_path / "pizza_steak_sushi"
zipfile_path = data_path / "pizza_steak_sushi.zip"

# If the image folder doesn't exist, download it and prepare it
if image_path.is_dir():
    print(f"{image_path} already exists")
else:
    print(f"Did not find {image_path} directory, creating one")
    image_path.mkdir(parents=True, exist_ok=True)

# Download data
with open(zipfile_path, "wb") as f:
    request = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")
    print("Downloading data")
    f.write(request.content)

# Unzip data
with zipfile.ZipFile(zipfile_path, "r") as zip_ref:
    print(f"Unzipping data")
    zip_ref.extractall(image_path)

# Remove zip file
os.remove(zipfile_path)

# To get the original dataset from torchvision.datasets
# food101_data_train = torchvision.datasets.Food101(root=image_path, split='train', download=True)
# food101_data_test = datasets.Food101(root=image_path, split='test', download=True)

Writing going_modular/get_data.py


In [3]:
!python going_modular/get_data.py

Did not find data/pizza_steak_sushi directory, creating one
Unzipping data


## 2. Use [Python's `argparse` module](https://docs.python.org/3/library/argparse.html) to be able to send the `train.py` custom hyperparameter values for training procedures.
- Add an argument for using a different:
    - Training/testing directory
    - Learning rate
    - Batch size
    - Number of epochs to train for
    - Number of hidden units in the TinyVGG model
- Keep the default values for each of the above arguments as what they already are (as in notebook 05).
- For example, you should be able to run something similar to the following line to train a TinyVGG model with a learning rate of 0.003 and a batch size of 64 for 20 epochs: `python train.py --learning_rate 0.003 --batch_size 64 --num_epochs 20`
- **Note:** Since `train.py` leverages the other scripts we created in section 05, such as, `model_builder.py`, `utils.py`, and `engine.py`, you'll have to make sure they're available to use too. You can find these in the [`going_modular` folder on the course GitHub](https://github.com/mrdbourke/pytorch-deep-learning/tree/main/going_modular/going_modular)

In [4]:
%%writefile going_modular/data_setup.py
"""
Contains functionality for creating PyTorch DataLoader
for image classification data
"""
import os

from torchvision import datasets, transforms
from torch.utils.data import DataLoader

NUM_WORKERS = os.cpu_count()

def create_dataloaders(
    train_dir: str,
    test_dir: str,
    transform: transforms.Compose,
    batch_size: int,
    num_workers: int=NUM_WORKERS
):
    """
    Creates training and testing dataloaders

    Takes in training and testing directory paths and turns them
    into PyTorch Dataset and then into PyTorch DataLoader

    Args:
        train_dir: Path to training directory
        test_dir: Path to testing directory
        transform: torchvision transforms to perform on training and testing data
        batch_size: Number of samples per batch in each of the DataLoader
        num_workers: An integer for number of workers per DataLoader
    
    Returns:
        A tuple of (train_dataloader, test_dataloader, class_names)
        where class_names is a list of the target classes
        
        Example usage:
            train_dataloader, test_dataloader, class_names = \
                create_dataloaders(train_dir=path/to/train_dir,
                                   test_dir=path/to/test_dir,
                                   transform=some_transform,
                                   batch_size=32,
                                   num_workers=2)
    """
    # Use ImageFolder to create datasets
    train_data = datasets.ImageFolder(train_dir, transform=transform)
    test_data = datasets.ImageFolder(test_dir, transform=transform)

    # Get class names
    class_names = train_data.classes

    # Turn images into dataloaders
    train_dataloader = DataLoader(dataset=train_data,
                                  batch_size=batch_size,
                                  shuffle=True,
                                  num_workers=num_workers,
                                  pin_memory=True)
    test_dataloader = DataLoader(dataset=test_data,
                                 batch_size=batch_size,
                                 shuffle=True,
                                 num_workers=num_workers,
                                 pin_memory=True)
    
    return train_dataloader, test_dataloader, class_names

Writing going_modular/data_setup.py


In [5]:
%%writefile going_modular/model_builder.py
"""
Contains PyTorch model code to instantiate a TinyVGG model.
"""
import torch
from torch import nn 

class TinyVGG(nn.Module):
  """Creates the TinyVGG architecture.

  Replicates the TinyVGG architecture from the CNN explainer website in PyTorch.
  See the original architecture here: https://poloclub.github.io/cnn-explainer/

  Args:
    input_shape: An integer indicating number of input channels.
    hidden_units: An integer indicating number of hidden units between layers.
    output_shape: An integer indicating number of output units.
  """
  def __init__(self, input_shape: int, hidden_units: int, output_shape: int) -> None:
      super().__init__()
      self.conv_block_1 = nn.Sequential(
          nn.Conv2d(in_channels=input_shape, 
                    out_channels=hidden_units, 
                    kernel_size=3, 
                    stride=1, 
                    padding=0),  
          nn.ReLU(),
          nn.Conv2d(in_channels=hidden_units, 
                    out_channels=hidden_units,
                    kernel_size=3,
                    stride=1,
                    padding=0),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size=2,
                        stride=2)
      )
      self.conv_block_2 = nn.Sequential(
          nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=0),
          nn.ReLU(),
          nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=0),
          nn.ReLU(),
          nn.MaxPool2d(2)
      )
      self.classifier = nn.Sequential(
          nn.Flatten(),
          # Where did this in_features shape come from? 
          # It's because each layer of our network compresses and changes the shape of our inputs data.
          nn.Linear(in_features=hidden_units*13*13,
                    out_features=output_shape)
      )

  def forward(self, x: torch.Tensor):
      x = self.conv_block_1(x)
      x = self.conv_block_2(x)
      x = self.classifier(x)
      return x

Writing going_modular/model_builder.py


In [6]:
%%writefile going_modular/engine.py
"""
Contains functions for training and testing a PyTorch model
"""
import torch

from tqdm.auto import tqdm
from typing import Dict, List, Tuple

def train_step(model: torch.nn.Module, 
               dataloader: torch.utils.data.DataLoader, 
               loss_fn: torch.nn.Module, 
               optimizer: torch.optim.Optimizer,
               device: torch.device) -> Tuple[float, float]:
    """Trains a PyTorch model for a single epoch.

    Turns a target PyTorch model to training mode and then
    runs through all of the required training steps (forward
    pass, loss calculation, optimizer step).

    Args:
    model: A PyTorch model to be trained.
    dataloader: A DataLoader instance for the model to be trained on.
    loss_fn: A PyTorch loss function to minimize.
    optimizer: A PyTorch optimizer to help minimize the loss function.
    device: A target device to compute on (e.g. "cuda" or "cpu").

    Returns:
    A tuple of training loss and training accuracy metrics.
    In the form (train_loss, train_accuracy). For example:

    (0.1112, 0.8743)
    """
    # Put model in train mode
    model.train()

    # Setup train loss and train accuracy values
    train_loss, train_acc = 0, 0

    # Loop through data loader data batches
    for batch, (X, y) in enumerate(dataloader):
        # Send data to target device
        X, y = X.to(device), y.to(device)

        # 1. Forward pass
        y_pred = model(X)

        # 2. Calculate  and accumulate loss
        loss = loss_fn(y_pred, y)
        train_loss += loss.item() 

        # 3. Optimizer zero grad
        optimizer.zero_grad()

        # 4. Loss backward
        loss.backward()

        # 5. Optimizer step
        optimizer.step()

        # Calculate and accumulate accuracy metric across all batches
        y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
        train_acc += (y_pred_class == y).sum().item()/len(y_pred)

    # Adjust metrics to get average loss and accuracy
    train_loss = train_loss / len(dataloader)
    train_acc = train_acc / len(dataloader)
    return train_loss, train_acc

def test_step(model: torch.nn.Module, 
              dataloader: torch.utils.data.DataLoader, 
              loss_fn: torch.nn.Module,
              device: torch.device) -> Tuple[float, float]:
    """Tests a PyTorch model for a single epoch.

    Turns a target PyTorch model to "eval" mode and then performs
    a forward pass on a testing dataset.

    Args:
    model: A PyTorch model to be tested.
    dataloader: A DataLoader instance for the model to be tested on.
    loss_fn: A PyTorch loss function to calculate loss on the test data.
    device: A target device to compute on (e.g. "cuda" or "cpu").

    Returns:
    A tuple of testing loss and testing accuracy metrics.
    In the form (test_loss, test_accuracy). For example:

    (0.0223, 0.8985)
    """
    # Put model in eval mode
    model.eval() 

    # Setup test loss and test accuracy values
    test_loss, test_acc = 0, 0

    # Turn on inference context manager
    with torch.inference_mode():
        # Loop through DataLoader batches
        for batch, (X, y) in enumerate(dataloader):
            # Send data to target device
            X, y = X.to(device), y.to(device)

            # 1. Forward pass
            test_pred_logits = model(X)

            # 2. Calculate and accumulate loss
            loss = loss_fn(test_pred_logits, y)
            test_loss += loss.item()

            # Calculate and accumulate accuracy
            test_pred_labels = test_pred_logits.argmax(dim=1)
            test_acc += ((test_pred_labels == y).sum().item()/len(test_pred_labels))

    # Adjust metrics to get average loss and accuracy
    test_loss = test_loss / len(dataloader)
    test_acc = test_acc / len(dataloader)
    return test_loss, test_acc

def train(model: torch.nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          test_dataloader: torch.utils.data.DataLoader,
          loss_fn: torch.nn.Module,
          optimizer: torch.optim.Optimizer,
          epochs: int,
          device: torch.device) -> Dict[str, List]:
    """Trains and tests a PyTorch model.

    Passes a target PyTorch models through train_step() and test_step()
    functions for a number of epochs, training and testing the model
    in the same epoch loop.

    Calculates, prints and stores evaluation metrics throughout.

    Args:
    model: A PyTorch model to be trained and tested.
    train_dataloader: A DataLoader instance for the model to be trained on.
    test_dataloader: A DataLoader instance for the model to be tested on.
    optimizer: A PyTorch optimizer to help minimize the loss function.
    loss_fn: A PyTorch loss function to calculate loss on both datasets.
    epochs: An integer indicating how many epochs to train for.
    device: A target device to compute on (e.g. "cuda" or "cpu").

    Returns:
    A dictionary of training and testing loss as well as training and
    testing accuracy metrics. Each metric has a value in a list for 
    each epoch.
    In the form: {train_loss: [...],
                    train_acc: [...],
                    test_loss: [...],
                    test_acc: [...]} 
    For example if training for epochs=2: 
                    {train_loss: [2.0616, 1.0537],
                    train_acc: [0.3945, 0.3945],
                    test_loss: [1.2641, 1.5706],
                    test_acc: [0.3400, 0.2973]} 
    """
    # Create empty results dictionary
    results = {"train_loss": [],
               "train_acc": [],
               "test_loss": [],
               "test_acc": []}

    # Loop through training and testing steps for a number of epochs
    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(model=model,
                                           dataloader=train_dataloader,
                                           loss_fn=loss_fn,
                                           optimizer=optimizer,
                                           device=device)
        test_loss, test_acc = test_step(model=model,
                                        dataloader=test_dataloader,
                                        loss_fn=loss_fn,
                                        device=device)
        
        # Print out what's happening
        print(f"Epoch: {epoch} | Train loss: {train_loss:.4f} | Train acc: {train_acc:.4f} | Test loss: {test_loss:.4f} | Test acc: {test_acc:.4f}")

        # Update results dictionary
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)

    # Return the filled results at the end of the epochs
    return results

Writing going_modular/engine.py


In [7]:
%%writefile going_modular/utils.py
"""
Contains various utility functions for PyTorch model training and saving.
"""
import torch
from pathlib import Path

def save_model(model: torch.nn.Module,
               target_dir: str,
               model_name: str):
    """Saves a PyTorch model to a target directory.

    Args:
    model: A target PyTorch model to save.
    target_dir: A directory for saving the model to.
    model_name: A filename for the saved model. Should include
        either ".pth" or ".pt" as the file extension.

    Example usage:
    save_model(model=model_0,
                target_dir="models",
                model_name="05_going_modular_tingvgg_model.pth")
    """
    # Create target directory
    target_dir_path = Path(target_dir)
    target_dir_path.mkdir(parents=True,
                          exist_ok=True)
    
    # Create model save path
    assert model_name.endswith(".pth") or model_name.endswith(".pt")
    model_save_path = target_dir_path / model_name

    # Save the model state_dict()
    print(f"[INFO] Saving model to: {model_save_path}")
    torch.save(obj=model.state_dict(),
               f=model_save_path)

Writing going_modular/utils.py


In [8]:
%%writefile going_modular/train.py
"""
Trains a PyTorch image classification model using device-agnostic code.
"""

import os
import torch
import argparse
import data_setup, engine, model_builder, utils

from torchvision import transforms

# Create a parser
parser = argparse.ArgumentParser(description="Get hyperparameters")

# Get arg for hyperparameters
parser.add_argument("--epochs", default=10, type=int, help="number of epochs to train the model")
parser.add_argument("--batch_size", default=32, type=int, help="number of samples per batch")
parser.add_argument("--hidden_units", default=10, type=int, help="number of hidden units in hidden layers")
parser.add_argument("--learning_rate", default=0.001, type=float, help="learning rate to train the model")
parser.add_argument("--train_dir", default="data/pizza_steak_sushi/train", type=str, help="directory file path to training data")
parser.add_argument("--test_dir", default="data/pizza_steak_sushi/test", type=str, help="directory file path to testing data")

# Get our arguments from the parser
args = parser.parse_args()

# Setup hyperparameters
EPOCHS = args.epochs
BATCH_SIZE = args.batch_size
HIDDEN_UNITS = args.hidden_units
LEARNING_RATE = args.learning_rate
print(f"[INFO] Training a model for {EPOCHS} epochs with batch size {BATCH_SIZE} using {HIDDEN_UNITS} hidden units and a learning rate of {LEARNING_RATE}")

# Setup directories
train_dir = args.train_dir
test_dir = args.test_dir
print(f"[INFO] Training data file path: {train_dir}")
print(f"[INFO] Testing data file path: {test_dir}")

# Setup target device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Create transforms
data_transform = transforms.Compose([
    transforms.Resize(size=(64, 64)),
    transforms.ToTensor()
])

# Create dataloaders using data_setup.py
train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(
    train_dir=train_dir,
    test_dir=test_dir,
    transform=data_transform,
    batch_size=BATCH_SIZE
)

# Create a model using model_builder.py
model = model_builder.TinyVGG(
    input_shape=3,
    hidden_units=HIDDEN_UNITS,
    output_shape=len(class_names)
).to(device)

# Set loss and optimizer
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters(),
                             lr=LEARNING_RATE)

# Start training using engine.py
engine.train(model=model,
             train_dataloader=train_dataloader,
             test_dataloader=test_dataloader,
             loss_fn=loss_fn,
             optimizer=optimizer,
             epochs=EPOCHS,
             device=device)

# Save the model using utils.py
utils.save_model(model=model,
                 target_dir="models",
                 model_name="05_going_modular_tingvgg_model.pth")

Writing going_modular/train.py


In [9]:
!python going_modular/train.py --epochs=20 --learning_rate=0.05 --batch_size=64

[INFO] Training a model for 20 epochs with batch size 64 using 10 hidden units and a learning rate of 0.05
[INFO] Training data file path: data/pizza_steak_sushi/train
[INFO] Testing data file path: data/pizza_steak_sushi/test
  0% 0/20 [00:00<?, ?it/s]Epoch: 0 | Train loss: 38.8435 | Train acc: 0.3182 | Test loss: 4.2773 | Test acc: 0.3835
  5% 1/20 [00:04<01:21,  4.30s/it]Epoch: 1 | Train loss: 1.9002 | Train acc: 0.3451 | Test loss: 1.1270 | Test acc: 0.2706
 10% 2/20 [00:06<00:55,  3.08s/it]Epoch: 2 | Train loss: 1.0985 | Train acc: 0.2901 | Test loss: 1.1403 | Test acc: 0.2614
 15% 3/20 [00:08<00:45,  2.69s/it]Epoch: 3 | Train loss: 1.1039 | Train acc: 0.3297 | Test loss: 1.1355 | Test acc: 0.2237
 20% 4/20 [00:11<00:42,  2.63s/it]Epoch: 4 | Train loss: 1.1019 | Train acc: 0.3333 | Test loss: 1.1263 | Test acc: 0.1861
 25% 5/20 [00:13<00:37,  2.47s/it]Epoch: 5 | Train loss: 1.1036 | Train acc: 0.3260 | Test loss: 1.1070 | Test acc: 0.3366
 30% 6/20 [00:16<00:38,  2.78s/it]Epoch: 6

## 3. Create a script to predict (such as `predict.py`) on a target image given a file path with a saved model.
- For example, you should be able to run the command `python predict.py` `some_image.jpeg` and have a trained PyTorch model predict on the image and return its prediction.
- To see example prediction code, check out the [predicting on a custom image section in notebook 04](https://www.learnpytorch.io/04_pytorch_custom_datasets/#113-putting-custom-image-prediction-together-building-a-function)
- You may also have to write code to load in a trained model.

In [10]:
%%writefile going_modular/predict.py

import torch
import torchvision
import argparse
import model_builder

# Creating a parser
parser = argparse.ArgumentParser()

# Get an image path
parser.add_argument("--image", help="target image to predict on")

# Get a model path
parser.add_argument("--model_path", default="models/05_going_modular_tingvgg_model.pth", help="target model to use for prediction")

args = parser.parse_args()

# Setup class names
class_names = ["pizza", "steak", "sushi"]

# Setup device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Get the image path
IMG_PATH = args.image
print(f"[INFO] Predicting on {IMG_PATH}")

# Function to load in the model
def load_model(filepath=args.model_path):
    # Need to use same hyperparameters as the saved model
    model = model_builder.TinyVGG(input_shape=3,
                                  hidden_units=10,
                                  output_shape=3).to(device)
    print(f"[INFO] Loading in model from: {filepath}")                              
    # Load in the saved model state dictionary from file                                  
    model.load_state_dict(torch.load(filepath))

    return model

# Function to load in model + predict on select image
def predict_on_image(image=IMG_PATH, filepath=args.model_path):
    # Load the model
    model = load_model(filepath)
    # Load in the image and turn it into torch.float32 (same type as model)
    image = torchvision.io.read_image(str(IMG_PATH)).type(torch.float32)

    # Preprocess the image to get it between 0 and 1
    image = image / 255.

    # Expand to 4 dims to include batch dimension [batch_size, color_channels, height, width]
    image = image.unsqueeze(0)

    # Resize the image to be on the same size as the model
    transform = torchvision.transforms.Resize(size=(64, 64), antialias=True)
    image = transform(image)

    # Predict on image
    model.eval()
    with torch.inference_mode():
        # Put image to target device
        image = image.to(device)

        # Get pred logits
        pred_logits = model(image)

        # Get pred probs
        pred_probs = torch.softmax(pred_logits, dim=1)

        # Get pred labels
        pred_label = torch.argmax(pred_probs, dim=1)
        pred_label_class = class_names[pred_label]

    print(f"[INFO] Pred class: {pred_label_class}, Pred prob: {pred_probs.max():.3f}")

# Indicating that when you pass the file, it'll be the main one
if __name__=="__main__":
    predict_on_image()

Writing going_modular/predict.py


In [11]:
!python going_modular/predict.py --image data/pizza_steak_sushi/test/steak/27415.jpg

[INFO] Predicting on data/pizza_steak_sushi/test/steak/27415.jpg
[INFO] Loading in model from: models/05_going_modular_tingvgg_model.pth
[INFO] Pred class: pizza, Pred prob: 0.357
